In [1]:
import pandas as pd
from numpy import nan
import pandas_gbq
import altair as alt
import matplotlib as mpl
import json

/home/pedro/Modelos/Faculdade/hackaton-fgv/venv/lib/python3.11/site-packages/geopandas/_compat.py:153: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  set_use_pygeos()


In [16]:
query = """SELECT *
    FROM datario.adm_cor_comando.ocorrencias
    WHERE data_particao > DATE("2023-06-01")""" # Coloquei um limite pra não gastar 3GB do google quando for puxar
chamados_rjcor = pandas_gbq.read_gbq(query, project_id = "hackaton-fgv")
chamados_rjcor

/home/pedro/Modelos/Faculdade/hackaton-fgv/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,id_pop,id_evento,bairro,data_inicio,data_fim,prazo,descricao,gravidade,latitude,longitude,status,tipo,data_particao
0,1,105018,São cristóvão,2023-10-26 21:24:27+00:00,2023-10-27 01:59:18+00:00,Curto,R. são luiz gonzaga - alt. lrg. da cancela - s...,Baixo,-22.900357,-43.224752,Fechado,Primario,2023-10-26
1,1,105016,Campo grande,2023-10-26 19:41:47+00:00,2023-10-26 19:51:15+00:00,Curto,R. prof. castilho alt. do nº 221 - campo grand...,Baixo,-22.909799,-43.564664,Fechado,Primario,2023-10-26
2,1,105009,Sampaio,2023-10-26 18:22:37+00:00,2023-10-26 18:24:57+00:00,Curto,R. vinte e quatro de maio - sampaio,Baixo,-22.902193,-43.261619,Fechado,Primario,2023-10-26
3,1,104460,Paciencia,2023-10-18 14:28:04+00:00,2023-10-18 15:34:09+00:00,Curto,Ponte rio/niteroi - alt. pórtico 18 - sent. ni...,Medio,-22.869978,-43.168039,Fechado,Primario,2023-10-18
4,1,104741,Del castilho,2023-10-23 11:55:24+00:00,2023-10-23 12:33:33+00:00,Curto,"Av. pastor martin luther king jr, alt. viaduto...",Baixo,-22.877472,-43.274988,Fechado,Primario,2023-10-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7385,25,7773,Santa teresa,2024-03-18 22:25:36+00:00,NaT,82147.0,"Tunel santa barbara, - santa teresa",Normal,-22.925607,-43.189977,Aberto,Primario,2024-03-18
7386,25,7759,Galeão,2024-03-18 16:24:27+00:00,NaT,77131.5,"Linha vermelha, alt. km 11 - galeao",Normal,-22.821851,-43.253935,Fechado,Primario,2024-03-18
7387,26,7747,Grajaú,2024-03-18 14:12:51+00:00,NaT,79323.0,"R. visconde de santa isabel, alt. r. barao do ...",Normal,-22.917550,-43.262130,Fechado,Primario,2024-03-18
7388,26,7733,Centro,2024-03-18 10:29:46+00:00,NaT,81338.0,"Av. alfred agache , alt. tunel marcello alenca...",Normal,-22.897306,-43.182677,Fechado,Primario,2024-03-18


In [17]:
chamados_rjcor["gravidade"].value_counts(dropna = False)

gravidade
Baixo     3864
Normal    2770
Medio      402
Média      339
Alta         8
None         4
Alto         3
Name: count, dtype: int64

In [21]:
gravidades = {
    "Normal": [255, 255, 0],
    "Média": [255, 165, 0],
    "Alta": [255, 0, 0],
    None: [120, 120, 120],
    "Baixo": [255, 255, 0],
    "Medio": [255, 165, 0],
    "Alto": [255, 0, 0]
}
chamados_rjcor["coloracao"] = chamados_rjcor.apply(lambda linha: gravidades[linha["gravidade"]], axis=1)
chamados_rjcor

,id_pop,id_evento,bairro,data_inicio,data_fim,prazo,descricao,gravidade,latitude,longitude,status,tipo,data_particao,coloracao
0,1,105018,São cristóvão,2023-10-26 21:24:27+00:00,2023-10-27 01:59:18+00:00,Curto,R. são luiz gonzaga - alt. lrg. da cancela - s...,Baixo,-22.900357,-43.224752,Fechado,Primario,2023-10-26,"[255, 255, 0]"
1,1,105016,Campo grande,2023-10-26 19:41:47+00:00,2023-10-26 19:51:15+00:00,Curto,R. prof. castilho alt. do nº 221 - campo grand...,Baixo,-22.909799,-43.564664,Fechado,Primario,2023-10-26,"[255, 255, 0]"
2,1,105009,Sampaio,2023-10-26 18:22:37+00:00,2023-10-26 18:24:57+00:00,Curto,R. vinte e quatro de maio - sampaio,Baixo,-22.902193,-43.261619,Fechado,Primario,2023-10-26,"[255, 255, 0]"
3,1,104460,Paciencia,2023-10-18 14:28:04+00:00,2023-10-18 15:34:09+00:00,Curto,Ponte rio/niteroi - alt. pórtico 18 - sent. ni...,Medio,-22.869978,-43.168039,Fechado,Primario,2023-10-18,"[255, 165, 0]"
4,1,104741,Del castilho,2023-10-23 11:55:24+00:00,2023-10-23 12:33:33+00:00,Curto,"Av. pastor martin luther king jr, alt. viaduto...",Baixo,-22.877472,-43.274988,Fechado,Primario,2023-10-23,"[255, 255, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7385,25,7773,Santa teresa,2024-03-18 22:25:36+00:00,NaT,82147.0,"Tunel santa barbara, - santa teresa",Normal,-22.925607,-43.189977,Aberto,Primario,2024-03-18,"[255, 255, 0]"
7386,25,7759,Galeão,2024-03-18 16:24:27+00:00,NaT,77131.5,"Linha vermelha, alt. km 11 - galeao",Normal,-22.821851,-43.253935,Fechado,Primario,2024-03-18,"[255, 255, 0]"
7387,26,7747,Grajaú,2024-03-18 14:12:51+00:00,NaT,79323.0,"R. visconde de santa isabel, alt. r. barao do ...",Normal,-22.917550,-43.262130,Fechado,Primario,2024-03-18,"[255, 255, 0]"
7388,26,7733,Centro,2024-03-18 10:29:46+00:00,NaT,81338.0,"Av. alfred agache , alt. tunel marcello alenca...",Normal,-22.897306,-43.182677,Fechado,Primario,2024-03-18,"[255, 255, 0]"


In [23]:
import pydeck as pdk

LAND_COVER = [[[-43.0, -23.5], [-43.0, -22.5], [-44, -22.5], [-44, -23.5]]]
INITIAL_VIEW_STATE = pdk.ViewState(
    latitude = -22.9035, 
    longitude = -43.4096, 
    zoom = 9.3, 
    max_zoom = 16, 
    pitch = 0, 
    bearing = 0
)

polygon = pdk.Layer(
    "PolygonLayer",
    #LAND_COVER,
    stroked=False,
    get_polygon="-",
    get_fill_color=[0, 0, 0, 20],
)

geojson = pdk.Layer(
    "ScatterplotLayer",
    chamados_rjcor[["latitude", "longitude", "coloracao"]],
    get_position = ["longitude", "latitude"],
    auto_highlight = True,
    get_radius = 50,
    get_fill_color = "coloracao",# "coloracao",
    pickable = True
)

r = pdk.Deck(layers=[geojson, polygon], initial_view_state=INITIAL_VIEW_STATE,
             map_style = "dark")

r.to_html("geojson_layer.html")